In [ ]:
import torch
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

max_seq_length = 2048
dtype = None
load_in_4bit = True

In [2]:
from unsloth import is_bfloat16_supported
max_seq_length = 512 
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    # fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    # gpu_memory_utilization = 0.7, # Reduce if out of memory
)

# model = FastLanguageModel.get_peft_model(
#     model,
#     r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["gate_proj", "up_proj", "down_proj",],
#     lora_alpha = lora_rank,
#     use_gradient_checkpointing = "unsloth", # Enable long context finetuning
#     random_state = 3407,
# )

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

model

Unsloth 2025.2.12 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(100352, 5120, padding_idx=100351)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [4]:
chat_prompt = """
### Instruction:
{}

### Question:
{}

### Choices:
{}

### Response:
{}"""

In [5]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = "Answer the multiple-choice question below based on the provided context. Your response has to be A, B, C, D, or E"
    inputs = examples["question"]
    choices = examples["choices"]
    outputs = examples["answerKey"]

    texts = []
    for input, choice, output in zip(inputs, choices, outputs):
        # Format choices in "A: ignore, B: enforce, ..." format
        choices_formatted = ", ".join([f"{label}: {text}" for label, text in zip(choice["label"], choice["text"])])
        # Create the prompt
        text = chat_prompt.format(instruction, input, choices_formatted, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

In [6]:
from datasets import load_dataset

ds = load_dataset("tau/commonsense_qa", split='train')
dataset = ds.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,574 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 65,536,000


Step,Training Loss
1,1.571700
2,1.595700
3,1.573500
4,1.585500
5,1.466100
6,1.488900
7,1.350700
8,1.328800
9,1.327600
10,1.328600


In [9]:
model.save_pretrained("phi_lora_commonsense_qa")
tokenizer.save_pretrained("phi_lora_commonsense_qa")

('phi_lora_commonsense_qa/tokenizer_config.json',
 'phi_lora_commonsense_qa/special_tokens_map.json',
 'phi_lora_commonsense_qa/vocab.json',
 'phi_lora_commonsense_qa/merges.txt',
 'phi_lora_commonsense_qa/added_tokens.json',
 'phi_lora_commonsense_qa/tokenizer.json')

In [10]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who is known for their work on classical conditioning?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]

response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

Pavlov

### Feedback:
Good job! You got it!

### Correct Feedback:
Pavlov

### Incorrect Feedback:
I'm sorry, you got it wrong. It's Pavlov.

### Solution:
Pavlov


In [11]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who is known for their work on classical conditioning?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]
print(chat_prompt.format(
            "",  # instruction
            "Who is known for their work on classical conditioning?",  # input
            "",  # output
        ))
print("=================")
print(decoded_output)
print("=================")
response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

### Instruction:


### Input:
Who is known for their work on classical conditioning?

### Response:

<|begin_of_text|>
### Instruction:


### Input:
Who is known for their work on classical conditioning?

### Response:
Pavlov

### Correct Answer:
Pavlov

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score
Pavlov

### Correct Answer:
Pavlov

### Score:
1

### Feedback:
Correct!

### Score:
1

#

In [12]:
print("=================")
print(decoded_output)

<|begin_of_text|>
### Instruction:


### Input:
Who is known for their work on classical conditioning?

### Response:
Pavlov

### Correct Answer:
Pavlov

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score:
1

### Feedback:
Correct!

### Score


In [13]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model_osloth", #model folder
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who is known for their work on classical conditioning?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]

response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

Pavlov

### Feedback:
Correct!
